In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/10 12:47:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
file_name = "home_sales_revised.csv"


In [12]:




spark.sparkContext.addFile(url)
sales_data = spark.read.option('header', 'true').csv(SparkFiles.get(file_name), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
# Create a temporary view of the DataFrame
sales_data.createOrReplaceTempView("sales_data")

# Show DataFrame
sales_data.show()


23/07/10 12:56:44 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

result = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM sales_data WHERE bedrooms = 4 GROUP BY date")
result.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2021-11-13|     358445.0|
|2019-05-08|     234979.5|
|2021-08-27|    275547.25|
|2021-01-27|    367520.57|
|2021-06-22|     297708.8|
|2021-10-11|    283388.33|
|2020-08-24|    301080.75|
|2022-03-28|    338544.78|
|2019-06-04|    306373.08|
|2021-12-18|     305264.3|
|2020-11-29|    318153.89|
|2020-01-21|     420576.0|
|2020-07-24|     293661.5|
|2021-11-25|    291762.27|
|2019-11-18|    290259.83|
|2019-09-22|     284864.5|
|2020-08-05|    243835.56|
|2019-11-01|    254751.38|
|2021-10-02|    303838.38|
|2020-08-29|     285457.3|
+----------+-------------+
only showing top 20 rows



In [16]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result1 = spark.sql("SELECT date_built AS year, ROUND(AVG(price), 2) AS average_price FROM sales_data WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY year")
result1.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [19]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result2 = spark.sql("Select date_built AS year, ROUND(AVG(price), 2) AS average_price From sales_data WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ")
result2.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2015|    297609.97|
|2013|    303676.79|
|2014|    298264.72|
|2012|    307539.97|
|2016|     293965.1|
|2010|    285010.22|
|2011|    276553.81|
|2017|    280317.58|
+----+-------------+



In [20]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result3 = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price, CASE WHEN AVG(price) >= 350000 THEN 'High' ELSE 'Low' END AS view FROM sales_data")
result3.show()




print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    314907.82| Low|
+-------------+----+

--- 0.3172428607940674 seconds ---


In [22]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales_data")


In [23]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_data')

True

In [27]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result_cached = spark.sql("SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM sales_data WHERE price >= 350000 GROUP BY view")
result_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.7406609058380127 seconds ---


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
sales_data.write.partitionBy('date_built').parquet('p_sales_data',mode='overwrite')

In [30]:
# 11. Read the formatted parquet data.
sales_data = spark.read.parquet('p_sales_data')

In [31]:
# 12. Create a temporary table for the parquet data.
sales_data.createOrReplaceTempView("parquet_sales_data")

In [32]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

result_parquet = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM parquet_sales_data WHERE price >= 350000 GROUP BY view")
result_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.5466351509094238 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("sales_data")


In [35]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached('sales_data'):
  print('home_sales is cached')
else:
  print('not cached')

not cached
